In [1]:
import multiprocessing as mp
import pandas as pd
import numpy as np
import glob
from datetime import date, datetime, time, timedelta
from collections import defaultdict
import os

# reduccion de datos: sin mediciones durante la noche

existen algunas estaciones que pierden datos. Esto se vuelve un problema grave si estos datos perdidos son en el periodo de 19 hrs de un dia y 9 hrs del dia siguiente. Estos datos (intervalo completo de 24 hrs desde las 19 hrs de un dia hasya las 19 hrs del dia siguiente) deben ser identificados y borrados


In [2]:
path = "../data/cleared"
dfs = []
dict_meta = {"region": [], "estacion": [], "codigo": []}
for file in glob.glob(path + "/*/*"):
    print(file + " " * 30, end="\r")
    file = file.replace("\\", "/")
    dict_meta["region"].append(file.split("/")[-2])
    dict_meta["estacion"].append(file.split("/")[-1].split(".")[0])
    df = pd.read_csv(file)
    dict_meta["codigo"].append(df["Codigo"][0])
    dfs.append(df)

In [3]:
df = pd.concat(dfs, ignore_index=True)
df = df.sort_values(by=['Codigo', 'Fecha', 'Hora'])
df = df.reset_index(drop=True)

In [7]:
df.columns

Index(['Codigo', 'Fecha', 'Hora', 'Temperatura', 'Humedad',
       'Direccion de Viento', 'Velocidad de Viento', 'Precipitacion',
       'Radiacion Solar', 'Presion Atmosferica'],
      dtype='object')

In [28]:
data = df.values
dummy = np.full((data.shape[0], 2), None)
data = np.append(data, dummy, axis=1)

In [29]:
data.shape

(5301924, 12)

In [35]:
def compute_datetime(data, ini, end):
    print("starting worker range [{}, {}]... ".format(ini, end), end="")
    ddtt = []
    displaced_date = []
    deltatime = timedelta(hours = 5)
    c = 0
    for i in range(ini, end):
        d = data[i][1]
        t = data[i][2]
        if " " in d:
            d = d.split(" ")[0]
            datm = datetime.strptime(d + " " + t, '%Y-%m-%d %H:%M:%S')
        elif "-" in d:
            datm = datetime.strptime(d + " " + t, '%Y-%m-%d %H:%M:%S')
        else:
            datm = datetime.strptime(d + " " + t, '%d/%m/%Y %H:%M:%S')
        
        data[i][10] = (datm + deltatime).__str__()
        data[i][11] = datm
#         displaced_date.append((datm + deltatime).date().__str__())
#         ddtt.append(datm)
        c += 1
#     out_q.put(c)
    print("done")
    
def run_compute_datetime(data, N):
    
    # obtenemos los indices
    r = data.shape[0] // N
    idxs = np.arange(N + 1) * r
    
    # lanzamos los workers
    print("starting workers ....")
    jobs = []
    for i in range(len(idxs)-1):
        ini = idxs[i]
        end = idxs[i+1]
        jobs.append(mp.Process(target=compute_datetime, args=(data, ini, end)))
        jobs[-1].start()
    print("waiting workers ...")
    for p in jobs:
        p.join()
        

# ddtt = []
# displaced_date = []
# # create datetime object and displaceit by -19 hours
# deltatime = timedelta(hours = 5)
# c = 0
# print(len(ddate))
# for d, t in zip(ddate, dtime):
    
#     if c % 10000 == 0:
#         print(c, end="\r")
#     if " " in d:
#         d = d.split(" ")[0]
#         datm = datetime.strptime(d + " " + t, '%Y-%m-%d %H:%M:%S')
#     elif "-" in d:
#         datm = datetime.strptime(d + " " + t, '%Y-%m-%d %H:%M:%S')
#     else:
#         datm = datetime.strptime(d + " " + t, '%d/%m/%Y %H:%M:%S')
#     displaced_date.append((datm + deltatime).date().__str__())
#     ddtt.append(datm)
#     c += 1
# compute_datetime(data, 0, 50, None)
# data
run_compute_datetime(data, 24)  
# df["datetime"] = ddtt
# df["displaced_date"] = displaced_date
# df.count()

starting workers ....
waiting workers ...
starting worker range [0, 220913]... done
starting worker range [441826, 662739]... done
starting worker range [662739, 883652]... done
starting worker range [220913, 441826]... done
starting worker range [1325478, 1546391]... done
starting worker range [883652, 1104565]... done
starting worker range [1104565, 1325478]... done
starting worker range [2209130, 2430043]... done
starting worker range [1546391, 1767304]... done
starting worker range [1767304, 1988217]... done
starting worker range [2430043, 2650956]... done
starting worker range [1988217, 2209130]... done
starting worker range [2650956, 2871869]... done
starting worker range [3092782, 3313695]... done
starting worker range [3313695, 3534608]... done
starting worker range [2871869, 3092782]... done
starting worker range [3534608, 3755521]... done
starting worker range [3755521, 3976434]... done
starting worker range [4197347, 4418260]... done
starting worker range [4639173, 4860086].

In [34]:
data

array([[13, '2010-06-11', '12:15:00', ..., 1008.0, '2010-06-11 17:15:00',
        datetime.datetime(2010, 6, 11, 12, 15)],
       [13, '2010-06-11', '12:30:00', ..., 1007.0, '2010-06-11 17:30:00',
        datetime.datetime(2010, 6, 11, 12, 30)],
       [13, '2010-06-11', '12:45:00', ..., 1007.0, '2010-06-11 17:45:00',
        datetime.datetime(2010, 6, 11, 12, 45)],
       ...,
       [450050, '2010-06-30', '02:30:00', ..., 1016.0, None, None],
       [450050, '2010-06-30', '02:45:00', ..., 1015.9, None, None],
       [450050, '2010-06-30', '03:00:00', ..., 1016.0, None, None]],
      dtype=object)

In [ ]:
datetime_arr = []
displaced_date_arr = []
qlen = rqueue.qsize()
while qlen > 0:
    ddtt_v, dispddtt_v = rqueue.get()
    datetime_arr.extend(ddtt_v)
    displaced_date_arr.extend(dispddtt_v)
    


In [16]:
a = 100
N = 20 
r = a // N
x = np.arange(N) * r
x

array([ 0,  5, 10, 15, 20, 25, 30, 35, 40, 45, 50, 55, 60, 65, 70, 75, 80,
       85, 90, 95])

In [7]:
df2v = df.drop_duplicates(subset=['Codigo', 'Fecha', 'Hora'], keep="first")
df2v.count()

Codigo                 5257742
Fecha                  5257742
Hora                   5257742
Temperatura            5257742
Humedad                5257742
Direccion de Viento    5257742
Velocidad de Viento    5257742
Precipitacion          5257742
Radiacion Solar        5257742
Presion Atmosferica    5257742
datetime               5257742
displaced_date         5257742
dtype: int64